SVD Proof-of-Concept Notebook
=============================

This notebook is just to make sure my SVD algorithm is correct, while I figure out how to export the C++ class to R for testing.

In [31]:
# Load modules and data
import numpy as np

L1 = 0

N = 1000
M = 100
K = 5

A = np.random.uniform(-1.0, 1.0, (M, N)) * 1000

U = np.random.uniform(-1.0, 1.0, (M, K))
V = np.zeros((K, N))

U_np, S_np, V_np = np.linalg.svd(A, full_matrices=False)
U_np = U_np * S_np[..., None, :]
print(U_np.shape)

TOL = 1e-9
MAX_ITER = 50

(100, 100)


In [32]:
# Error measurement
def mse(A, U, V):
    D = A - np.matmul(U, V)
    D = D ** 2
    return np.sum(D) / (D.shape[0] * D.shape[1])

# L2 norm
def norm(x):
    return np.sqrt(np.sum(x * x))

In [33]:
# Solve each K in turn 
print(f"Initial model error:\t{mse(A, U, V)}")
for k in range(K):
    for iter in range(MAX_ITER):
        if k == 0:
            # Update V
            a = np.matmul(U[:, 0].transpose(), U[:, 0])
            b = np.matmul(A.transpose(), U[:, 0]) - L1
            
            V[0, :] = b / a

            # Scale V
            V[0, :] = V[0, :] / norm(V[0, :])

            # Update U
            a =  np.matmul(V[0, :], V[0, :].transpose())
            b = np.matmul(A, V[0, :].transpose()) - L1

            U[:, 0] = b / a

            # Scale U
            d =  norm(U[0, :])
            U[:, 0] = U[:, 0] / d

        else:
            # Update V
            a = np.matmul(U[:, :k+1].transpose(), U[:, k])
            b = np.matmul(U[:, k].transpose(), A) - L1

            for n in range(N):
                for i in range(k):
                    b[n] -= a[i] * V[i,n]

            V[k, :] = b / a[k]

            # Scale V
            V[k, :] = V[k, :] / norm(V[k, :])

            # Update U
            a =  np.matmul(V[k, :], V[:k+1, :].transpose())
            b = np.matmul(A, V[k, :].transpose()) - L1
               
            for m in range(M):
                for i in range(k):
                    b[m] -= a[i] * U[m,i]

            U[:, k] = b / a[k]

            # Scale U
            d =  norm(U[:, k])
            U[:, k] = U[:, k] / d

    # 'Unscale' U
    U[:, k] = U[:, k] * d

    print(f"Model error after K={k+1}:\t{mse(A, U, V)}")
    print(f"numpy SVD implementation error: {mse(A, U_np[:, :k+1], V_np[:k+1, :])}")
    print()

Initial model error:	333657.1378083372
Model error after K=1:	327985.201266717
numpy SVD implementation error: 327981.25055388844

Model error after K=2:	322489.3834138897
numpy SVD implementation error: 322470.0390760356



ValueError: operands could not be broadcast together with shapes (2,) (2,1000) 